# Import

In [152]:
import pandas as pd
import numpy as np
import datetime

## Transform the .json into csv then dataframe  

In [153]:
data = pd.read_json("../data/final_dataset.json")

df = pd.DataFrame(data)
data_csv1 = df.to_csv("../data/dataset1.csv")

df.head()

,Url,BathroomCount,BedroomCount,ConstructionYear,Country,District,Fireplace,FloodingZone,Furnished,Garden,...,RoomCount,ShowerCount,StateOfBuilding,SubtypeOfProperty,SurfaceOfPlot,SwimmingPool,Terrace,ToiletCount,TypeOfProperty,TypeOfSale
2,https://www.immoweb.be/en/classified/apartment...,1.0,1,1969.0,Belgium,Brugge,NaN,None,NaN,NaN,...,1.0,0.0,GOOD,flat_studio,NaN,NaN,1.0,1.0,2,residential_sale
6,https://www.immoweb.be/en/classified/house/for...,6.0,13,1920.0,Belgium,Tournai,NaN,None,0.0,NaN,...,31.0,NaN,GOOD,apartment_block,130.0,NaN,NaN,5.0,1,residential_sale
8,https://www.immoweb.be/en/classified/house/for...,2.0,4,2008.0,Belgium,Brugge,NaN,NON_FLOOD_ZONE,1.0,NaN,...,NaN,0.0,GOOD,house,0.0,NaN,NaN,2.0,1,residential_sale
10,https://www.immoweb.be/en/classified/house/for...,1.0,4,NaN,Belgium,Veurne,NaN,None,NaN,1.0,...,NaN,NaN,TO_BE_DONE_UP,house,170.0,0.0,1.0,2.0,1,residential_sale
11,https://www.immoweb.be/en/classified/apartment...,0.0,2,1972.0,Belgium,Hasselt,NaN,NON_FLOOD_ZONE,NaN,NaN,...,1.0,0.0,AS_NEW,apartment,NaN,NaN,1.0,1.0,2,residential_sale


In [154]:
df.columns



Index(['Url', 'BathroomCount', 'BedroomCount', 'ConstructionYear', 'Country',
       'District', 'Fireplace', 'FloodingZone', 'Furnished', 'Garden',
       'GardenArea', 'Kitchen', 'LivingArea', 'Locality', 'MonthlyCharges',
       'NumberOfFacades', 'PEB', 'PostalCode', 'Price', 'PropertyId',
       'Province', 'Region', 'RoomCount', 'ShowerCount', 'StateOfBuilding',
       'SubtypeOfProperty', 'SurfaceOfPlot', 'SwimmingPool', 'Terrace',
       'ToiletCount', 'TypeOfProperty', 'TypeOfSale'],
      dtype='object')

In [155]:
df.shape

(118714, 32)

### Start cleaning 
- No duplicates
- No blank spaces (ex: `" I love python "` => `"I love python"`)
- No errors
- No empty values

In [156]:
df.drop_duplicates(inplace=True)
df = df.drop(columns = ["Country", "Fireplace"])
df.shape

(118714, 30)

### Remove the house built more than 10 year in the future 

In [157]:
year_threshold = datetime.datetime.today().year + 10
        
df = df.loc[(df['ConstructionYear'].isnull()) | (df['ConstructionYear'] <= year_threshold)]
df = df.loc[~((df['GardenArea'] > 0) & (df['Garden'] == False))]
df = df.loc[df['LivingArea'] >= 9]
# df = df.loc[df['TypeOfSale']]

df.to_csv("../data/dataset1.csv")

#### Fill some empty values 

In [158]:

default = {'numeric': 0,'string': "null",}


# Update specific columns with appropriate empty values
numeric_columns = ['BathroomCount', 'BedroomCount', 'ConstructionYear', 'GardenArea', 
                   'LivingArea', 'MonthlyCharges', 'NumberOfFacades', 'Price', 
                   'RoomCount', 'ShowerCount', 'SurfaceOfPlot', 'SwimmingPool', 
                   'Terrace', 'ToiletCount', 'Furnished', 'Garden']


fill_values = {col: default['numeric'] if col in numeric_columns else default['string'] for col in df.columns}
fill_values['ConstructionYear'] = "null"  

df.fillna(value=fill_values, inplace=True)

data_csv1 = df.to_csv("../data/dataset1.csv")

# Strip values of blank space 

In [159]:
for i in df.columns :
    #Check datatype for each columns
    if df[i].dtype == "str" : 
        df[i] = df[i].map(str.strip)